In [1]:
import numpy as np
from functools import partial
import ipywidgets as widgets
from IPython.display import display

In [2]:
#pydrake imports
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptions, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions
from pydrake.all import PiecewisePolynomial, InverseKinematics, Sphere, Rgba, RigidTransform, RotationMatrix, IrisInConfigurationSpace
import time
import pydrake
from pydrake.all import (SceneGraphCollisionChecker, 
                         StartMeshcat, 
                         RobotDiagramBuilder,
                         ProcessModelDirectives,
                         LoadModelDirectives,
                         MeshcatVisualizer,
                        DiagramBuilder,
                         MultibodyPlant,
                         AddMultibodyPlantSceneGraph,
                         PiecewisePolynomial,
                        Parser)
from pydrake.all import GeometrySet, CollisionFilterDeclaration
from scipy.special import comb
import matplotlib.pyplot as plt
import pydrake.multibody.rational as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
http://localhost:7000/
import time
from pydrake.geometry.optimization_dev import CspaceFreePath
from pydrake.all import Role, MeshcatVisualizerParams
from pydrake.all import InverseDynamicsController, TrajectorySource,StateInterpolatorWithDiscreteDerivative
from copy import deepcopy


In [3]:
from pydrake.examples import ManipulationStation


In [4]:
meshcat = StartMeshcat()


INFO:drake:Meshcat listening for connections at http://localhost:7000


In [5]:
from pydrake.all import LeafSystem
class ConvertSToQ(LeafSystem):
    def __init__(self, q_star, Ratfk):
        super().__init__()  
        self.s_port = self.DeclareVectorInputPort("s", q_star.shape[0])
        self.DeclareVectorOutputPort("q", q_star.shape[0], self.CalcOutput)
        self.q_star = q_star
        self.Ratfk = Ratfk
        
    def CalcOutput(self, context, output):
        s = self.s_port.Eval(context)
        q = self.Ratfk.ComputeQValue(s, self.q_star)
        output.set_value(q)

In [6]:
meshcat.Delete()
builder = DiagramBuilder()
iiwa_directive_file = "7_dof_iiwa_directive.yaml"
shelf_directive_file = "7_dof_iiwa_shelves_directive.yaml"
time_step = 0.002
# Build one plant to model the world
sim_plant, sim_scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
sim_parser = Parser(sim_plant)
iiwa_directive = LoadModelDirectives(iiwa_directive_file)
shelf_directive = LoadModelDirectives(shelf_directive_file)
iiwa_model_info = ProcessModelDirectives(iiwa_directive, sim_plant, sim_parser)
shelves_model_info = ProcessModelDirectives(shelf_directive, sim_plant, sim_parser)
## Add any other models you want the robot to manipulate here.
sim_plant.Finalize()
meshcat_params = MeshcatVisualizerParams()
visualizer = MeshcatVisualizer.AddToBuilder(builder, sim_scene_graph, meshcat, meshcat_params)



controller_plant = MultibodyPlant(time_step)
ProcessModelDirectives(iiwa_directive, controller_plant, Parser(controller_plant))
controller_plant.Finalize()
Ratfk = RationalForwardKinematics(controller_plant)

num_positions = controller_plant.num_positions()

# Add an inverse dynamics controller which uses the controller_plant.
inv_dynamics_controller = builder.AddSystem(
    InverseDynamicsController(
        controller_plant,
        kp=[1000] * num_positions,
        ki=[1] * num_positions,
        kd=[20] * num_positions,
        has_reference_acceleration=False,
    )
)

inv_dynamics_controller.set_name("inv_dynamics_controller")
builder.Connect(
    inv_dynamics_controller.get_output_port_control(),
    sim_plant.get_actuation_input_port(),
)

iiwa = sim_plant.GetModelInstanceByName("iiwa")
builder.Connect(
    sim_plant.get_state_output_port(iiwa), 
    inv_dynamics_controller.get_input_port_estimated_state()
)

desired_state_from_position = builder.AddSystem(
    StateInterpolatorWithDiscreteDerivative(
        controller_plant.num_positions(),
        time_step,
        suppress_initial_transient=True,
    )
)

builder.Connect(
    desired_state_from_position.get_output_port(),
    inv_dynamics_controller.get_input_port_desired_state(),
)

s_to_q = builder.AddSystem(ConvertSToQ(np.zeros(controller_plant.num_positions()), RationalForwardKinematics(controller_plant)))
s_trajectory_source = builder.AddSystem(
    TrajectorySource(PiecewisePolynomial(np.zeros(controller_plant.num_positions())))
)
builder.Connect(s_trajectory_source.get_output_port(),
               s_to_q.get_input_port())
builder.Connect(s_to_q.get_output_port(),
               desired_state_from_position.get_input_port())


collision_filter_manager = sim_scene_graph.collision_filter_manager()
inspector = sim_scene_graph.model_inspector()
shelf_and_ground_col_bodies = []
for model_index in shelves_model_info:
    for body_index in sim_plant.GetBodyIndices(model_index.model_instance):
        body = sim_plant.get_body(body_index)
        shelf_and_ground_col_bodies += sim_plant.GetCollisionGeometriesForBody(body)


diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
sim_plant_context = sim_plant.GetMyContextFromRoot(diagram_context)
sim_scene_graph_context = sim_scene_graph.GetMyContextFromRoot(diagram_context)


meshcat.SetProperty("/Axes", "visible", False)
meshcat.SetProperty("/Grid", "visible", False)
meshcat.SetProperty("/Background", "visible", False)
meshcat.SetCameraPose(
    camera_in_world = np.array([1.5,1.75,1.5]),
    target_in_world = np.array([0,0,0])
)

diagram.ForcedPublish(diagram_context)

q_star = np.zeros(controller_plant.num_positions())
inspector = sim_scene_graph.model_inspector()
original_geometry_id_props = {
    geometry_id: inspector.GetProperties(geometry_id, Role.kIllustration) for geometry_id in inspector.GetAllGeometryIds()
}



In [7]:
from scipy.interpolate import CubicSpline
waypoints_q =  np.array([[-2.06706, -0.3944 ,  1.43294,  0.9056 ,  0.53294, -0.8944 ,
        0.74567],
[-2.26706, -0.3944 ,  1.43294,  0.9056 ,  0.73294, -0.9944 ,
        1.04567],
[-2.96706, -0.344 ,  2.83294,  0.9056 ,  0.73294, -0.9944 ,
        1.04567],
[-1.06706,  0.1056 ,  2.43294,  0.9056 ,  0.73294,  0.2056 ,
        1.04567],
[-1.06706,  0.1056 ,  2.43294,  1.1056 ,  1.73294,  0.2056 ,
        0.0567],
[-1.56706,  0.1056 ,  2.43294,  1.2056 ,  1.73294,  0.8056 ,
       -0.25433]])

# waypoints_q[1,:]=np.array([
#     -2.16706, -0.3944 ,  1.43294,  0.9056 ,  0.73294, -1.0944, 1.04567
# ])

t = np.linspace(0, 1, waypoints_q.shape[0])
coord_splines = [CubicSpline(t,waypoints_q[:,i]) for i in range(7)]
t_refined = np.linspace(0,1,30)
waypoints_q_refined = [[c(t) for c in coord_splines]for t in t_refined] 



waypoints_s = np.array([Ratfk.ComputeSValue(w, np.zeros(7)) for w in waypoints_q_refined])


waypoints_q_col = waypoints_q.copy()

waypoints_q_col[1,:]=np.array([-2.22706, -0.3944 ,  1.43294,  0.9056 ,  0.73294, -0.9944 ,
        1.04567])
# waypoints_q_col[1,:]=np.array([
#     -2.06706, -0.3944 ,  1.33294,  0.9056 ,  0.73294, -0.5944 ,  0.54567
# ])
# waypoints_q_col[2,:]=np.array([
#     -2.06706, -0.3944 ,  1.33294,  0.9056 ,  0.73294, -0.5944 ,
#         0.54567
# ])

t = np.linspace(0, 1, waypoints_q_col.shape[0])
coord_splines = [CubicSpline(t,waypoints_q_col[:,i]) for i in range(7)]
t_refined = np.linspace(0,1,30)
waypoints_q_col_refined = [[c(t) for c in coord_splines]for t in t_refined] 

t = np.linspace(0, 1, waypoints_q_col.shape[0])
coord_splines = [CubicSpline(t,waypoints_q_col[:,i]) for i in range(7)]
t_refined = np.linspace(0,1,30)
waypoints_q_col_refined = [[c(t) for c in coord_splines]for t in t_refined] 


waypoints_s_col = np.array([Ratfk.ComputeSValue(w, np.zeros(7)) for w in waypoints_q_col_refined])
#np.array([-2.0706, -0.3244 ,  1.43294,  0.8756 ,  0.73294, -0.9944 ,
#      1.04567])

breaks = np.linspace(0, 1, waypoints_s.shape[0])
samples = waypoints_s

traj_time = 3

# traj_safe = PiecewisePolynomial.CubicShapePreserving(breaks,samples.T)
# polys_safe = np.array([traj_safe.getPolynomialMatrix(i) for i in breaks[:-1]]).squeeze().T
traj_safe = PiecewisePolynomial.CubicWithContinuousSecondDerivatives(breaks,samples.T)
traj_safe.ScaleTime(traj_time)
polys_safe = np.array([traj_safe.getPolynomialMatrix(i) for i in range(len(breaks)-1)]).squeeze().T


breaks = np.linspace(0, 1, waypoints_s_col.shape[0])
samples = waypoints_s_col
# traj_unsafe = PiecewisePolynomial.CubicShapePreserving(breaks,samples.T)
# polys_unsafe = np.array([traj_safe.getPolynomialMatrix(i) for i in breaks[:-1]]).squeeze().T
traj_unsafe = PiecewisePolynomial.CubicWithContinuousSecondDerivatives(breaks,samples.T)
traj_unsafe.ScaleTime(traj_time)

polys_unsafe = np.array([traj_safe.getPolynomialMatrix(i) for i in range(len(breaks)-1)]).squeeze().T



In [8]:
# q = np.zeros(plant.num_positions())
# ik = InverseKinematics(plant, plant_context)
# collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.001)
# def eval_cons(q, c, tol):
#     return 1-1*float(c.evaluator().CheckSatisfied(q, tol))
# diagram_col_context = diagram.CreateDefaultContext()
# plant_col_context = diagram.GetMutableSubsystemContext(sim_plant,
#                                                        diagram_col_context)
# scene_graph_col_context = diagram.GetMutableSubsystemContext(
#         scene_graph, diagram_col_context
#     )
query_port = sim_scene_graph.get_query_output_port()

def SetSimPlantIiwaPosition(q, context = sim_plant_context):
    cur_q = sim_plant.GetPositions(context)
    cur_q[:controller_plant.num_positions()] = q
    sim_plant.SetPositions(context, cur_q)
    
def SetSimPlantIiwaPositionS(s, context = sim_plant_context):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    SetSimPlantIiwaPosition(q, context)
    
def check_collision_q_by_query(q):
    SetSimPlantIiwaPosition(q)
    query_object = query_port.Eval(sim_scene_graph_context)
    return 1 if query_object.HasCollisions() else 0
    

def check_collision_s_by_query(s):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_query(q)


    
col_func_handle = check_collision_q_by_query#partial(eval_cons, c=collision_constraint, tol=0.01)
# col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)

col_shunk_col =  Rgba(0.8, 0.0, 0, 0.5)    
col_shunk_free =  Rgba(0.0, 0.8, 0.5, 0.5)   


def showres_s(s_vis):
    showres(Ratfk.ComputeQValue(s_vis, np.zeros(7)))

def showres(qvis):
    SetSimPlantIiwaPosition(qvis)
    
    col = col_func_handle(qvis)
    
    diagram.ForcedPublish(diagram_context)



In [9]:
import C_Iris_Examples.visualization_utils as vis_utils
q_star = np.zeros(controller_plant.num_positions())
vis_bundle = vis_utils.VisualizationBundle(
    diagram, diagram_context, sim_plant, sim_plant_context,
    Ratfk, meshcat, q_star
)
pos_set_fun_s = SetSimPlantIiwaPositionS

num_points = int(1e4)
safe_color=Rgba(0,1,0,1)
vis_utils.visualize_s_space_trajectory(
    vis_bundle, traj_safe,
    sim_plant.GetBodyByName("body"), #shunk,
    "/safe_traj",
    vis_utils.TrajectoryVisualizationOptions(start_size = 0.001,
                                            end_size=0.001,
                                            start_color=safe_color,
                                            end_color=safe_color,
                                            path_color=safe_color,
                                            num_points = num_points),
    pos_set_fun_s
)

unsafe_color = Rgba(1,0,0,1)
vis_utils.visualize_s_space_trajectory(
    vis_bundle, traj_unsafe,
    sim_plant.GetBodyByName("body"), #shunk,
    "/unsafe_traj",
    vis_utils.TrajectoryVisualizationOptions(start_size = 0.001,
                                            end_size=0.001,
                                            start_color=unsafe_color,
                                            end_color=unsafe_color,
                                            path_color=unsafe_color,
                                            num_points = num_points),
    pos_set_fun_s
)

In [10]:
from pydrake.all import MakePhongIllustrationProperties, RoleAssign
def SetDiffuseByBody(plant, scene_graph, body_index, rgba,
               scene_graph_context,  geometry_name = None):
    inspector = scene_graph.model_inspector()
    frame_id = plant.GetBodyFrameIdIfExists(body_index)
    if frame_id is not None:
        for geometry_id in inspector.GetGeometries(frame_id,
                                                   Role.kIllustration):
            if geometry_name is not None:
                if inspector.GetName(geometry_id) != geometry_name:
                    continue
            props = inspector.GetProperties(geometry_id, Role.kIllustration)
            if (props is None or not props.HasProperty("phong", "diffuse")):
                continue
            new_props = MakePhongIllustrationProperties(rgba)
            if scene_graph_context is None:
                scene_graph.AssignRole(plant.get_source_id(), geometry_id,
                                       new_props, RoleAssign.kReplace)
            else:
                scene_graph.AssignRole(scene_graph_context,
                                       plant.get_source_id(), geometry_id,
                                       new_props, RoleAssign.kReplace)
def SetDiffuseByGeometryId(plant, scene_graph, geometry_id_trigger, rgba,
               scene_graph_context,  geometry_name = None):
    inspector = scene_graph.model_inspector()
    frame_id = inspector.GetFrameId(geometry_id_trigger)
    for geometry_id in inspector.GetGeometries(frame_id,
                                                Role.kIllustration):
    
        props = inspector.GetProperties(geometry_id, Role.kIllustration)
        if (props is None or not props.HasProperty("phong", "diffuse")):
            continue
        new_props = MakePhongIllustrationProperties(rgba)
        if scene_graph_context is None:
            scene_graph.AssignRole(plant.get_source_id(), geometry_id,
                                   new_props, RoleAssign.kReplace)
        else:
            scene_graph.AssignRole(scene_graph_context,
                                   plant.get_source_id(), geometry_id,
                                   new_props, RoleAssign.kReplace)
            
def ResetDiffuseByBody(plant, scene_graph, body_index,
               scene_graph_context,  geometry_name = None):
    inspector = scene_graph.model_inspector()
    frame_id = plant.GetBodyFrameIdIfExists(body_index)
    if frame_id is not None:
        for geometry_id in inspector.GetGeometries(frame_id,
                                                   Role.kIllustration):
            if geometry_name is not None:
                if inspector.GetName(geometry_id) != geometry_name:
                    continue
            props = original_geometry_id_props[geometry_id]
            if (props is None or not props.HasProperty("phong", "diffuse")):
                continue
            if scene_graph_context is None:
                scene_graph.AssignRole(plant.get_source_id(), geometry_id,
                                       props, RoleAssign.kReplace)
            else:
                scene_graph.AssignRole(scene_graph_context,
                                       plant.get_source_id(), geometry_id,
                                       props, RoleAssign.kReplace)

def ResetDiffuseByGeometryId(plant, scene_graph, geometry_id_trigger,
               scene_graph_context,  geometry_name = None):
    inspector = scene_graph.model_inspector()
    frame_id = inspector.GetFrameId(geometry_id_trigger)
    for geometry_id in inspector.GetGeometries(frame_id,
                                                Role.kIllustration):
    
        props = original_geometry_id_props[geometry_id]
        if (props is None or not props.HasProperty("phong", "diffuse")):
            continue
        if scene_graph_context is None:
            scene_graph.AssignRole(plant.get_source_id(), geometry_id,
                                   props, RoleAssign.kReplace)
        else:
            scene_graph.AssignRole(scene_graph_context,
                                   plant.get_source_id(), geometry_id,
                                   props, RoleAssign.kReplace)

In [11]:
SetDiffuseByBody(sim_plant, sim_scene_graph,
          sim_plant.GetBodyByName("body").index(),
          np.array((1,0,0,1)),
          sim_scene_graph_context)
diagram.ForcedPublish(diagram_context)

In [12]:
ResetDiffuseByBody(sim_plant, sim_scene_graph,
          sim_plant.GetBodyByName("body").index(),
          sim_scene_graph_context)
diagram.ForcedPublish(diagram_context)

## Now simulate the trajectory

In [13]:
from pydrake.all import Simulator
def simulate_s_traj(s_traj, end_time = None, real_time_rate = 0):
    s_trajectory_source.UpdateTrajectory(s_traj)
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator_context = simulator.get_mutable_context()
    plant_context = sim_plant.GetMyMutableContextFromRoot(simulator_context)
    scene_graph_context = sim_scene_graph.GetMyMutableContextFromRoot(simulator_context)
    visualizer_context = visualizer.GetMyContextFromRoot(simulator_context)
    inspector = sim_scene_graph.model_inspector()
    SetSimPlantIiwaPositionS(s_traj.value(0).flatten(),
                            context = plant_context)
    

    def highlight_callback(context):
        query_output = sim_scene_graph.get_query_output_port().Eval(scene_graph_context)
        collision_pairs = query_output.ComputePointPairPenetration()
        for pair in collision_pairs:
            geom1, geom2 = pair.id_A, pair.id_B
            if not (geom1 in shelf_and_ground_col_bodies and geom2 in shelf_and_ground_col_bodies):
                for geom in (geom1, geom2):
                    SetDiffuseByGeometryId(sim_plant, sim_scene_graph,
                      geom,
                      np.array((1,0,0,1)),
                      sim_scene_graph_context)
        sim_scene_graph.ForcedPublish(scene_graph_context)
        visualizer.ForcedPublish(visualizer_context)
        
        
            
    
    simulator.set_monitor(highlight_callback)

    meshcat.StartRecording()
    if end_time is None:
        end_time = s_traj.end_time()
    simulator.set_target_realtime_rate(real_time_rate)
    simulator.AdvanceTo(end_time)
    meshcat.StopRecording()
    animation = meshcat.get_mutable_recording()
    animation.set_autoplay(False)
    meshcat.PublishRecording()
    


In [90]:
def playback_traj_n_times(traj, n, sleep_time = 2, real_time_rate = 0):
    dummy_traj = traj.Clone()
    new_traj = traj.Clone()
    for idx in range(n):
        new_traj.ConcatenateInTime(PiecewisePolynomial.ZeroOrderHold(
            [new_traj.end_time(), new_traj.end_time()+2],
            np.hstack([new_traj.value(new_traj.end_time()), 
                                       new_traj.value(new_traj.end_time())])
        ))
        dummy_traj.ReverseTime()
        dummy_traj.shiftRight(new_traj.end_time() - dummy_traj.start_time())
        new_traj.ConcatenateInTime(dummy_traj)
    new_traj.shiftRight(4)
    simulate_s_traj(new_traj,real_time_rate=real_time_rate)


In [97]:
playback_traj_n_times(traj_safe, 5, 2, 1)
with open("safe_7_dof_traj.html", "w+") as f:
    f.write(meshcat.StaticHtml())

In [98]:
playback_traj_n_times(traj_unsafe, 5, 2, 1)
with open("unsafe_7_dof_traj.html", "w+") as f:
    f.write(meshcat.StaticHtml())

In [16]:
# query_output = sim_scene_graph.get_query_output_port().Eval(sim_scene_graph_context)
# collision_pairs = query_output.ComputePointPairPenetration()
# for pair in collision_pairs:
#     geom1, geom2 = pair.id_A, pair.id_B
#     if not (geom1 in shelf_and_ground_col_bodies and geom2 in shelf_and_ground_col_bodies):
#         for geom in (geom1, geom2):
#             ResetDiffuseByGeometryId(sim_plant, sim_scene_graph,
#               geom,
#     #           np.array((1,0,0,1)),
#               sim_scene_graph_context)
# diagram.ForcedPublish(diagram_context)

In [17]:
raise ValueError()

ValueError: 

In [ ]:
# 
# plane_order = 2
# max_degree = 3
# t0 = time.time()
# cspace_free_path = CspaceFreePath(
#     plant,
#     scene_graph,
#     q_star,
#     maximum_path_degree=max_degree,
#     plane_order=plane_order,
# )
# t1 = time.time()
# print(f"Time to build collision checker {t1-t0}")

In [ ]:
len(cspace_free_path.separating_planes())

In [ ]:
# for plane in cspace_free_path.separating_planes():
#     pos_body_idx = plane.positive_side_geometry.body_index()
#     neg_body_idx = plane.negative_side_geometry.body_index()
#     pos_body_name = plant.get_body(pos_body_idx).name()
#     neg_body_name = plant.get_body(neg_body_idx).name()
#     print(pos_body_name, neg_body_name)
print(len(cspace_free_path.separating_planes()))

In [ ]:
cert_options = CspaceFreePath.FindSeparationCertificateGivenPathOptions()
cert_options.terminate_segment_certification_at_failure = False
cert_options.num_threads = -1
cert_options.verbose = False
cert_options.solver_id = MosekSolver.id()
cert_options.solver_options = SolverOptions()
cert_options.terminate_path_certification_at_failure = False

In [ ]:
q = waypoints_q[1,:].copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    showres(q)    
sliders = []
for i in range(7):
    q_low = sim_plant.GetPositionLowerLimits()[i]
    q_high = sim_plant.GetPositionUpperLimits()[i]
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=q[i], description=f"q{i}"))

    
scaler = 1 #np.array([0.8, 1., 0.8, 1, 0.8, 1, 0.8]) #do you even geometry bro?
q_min = np.array(q_low)*scaler
q_max = np.array(q_high)*scaler
q_diff = q_max-q_min

idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

display(sliders[0])
for slider in sliders:
    display(slider)

In [ ]:
print(repr(
    sim_plant.GetPositions(sim_plant_context)[:7]
))

In [ ]:
t0 = time.time()
stats, res = cspace_free_path.FindSeparationCertificateGivenPath(polys,
                                                                 set(),
                                                                 cert_options)
t1 = time.time()
print(f"Time to certify path {t1-t0}")

In [ ]:
for s in stats:
    print(s.certified_safe())
    

In [ ]:
print(f"Total safe solver time = {sum([s.total_time_to_solve_progs()/20 for s in stats])}")

In [ ]:
t0 = time.time()
stats, res = cspace_free_path.FindSeparationCertificateGivenPath(polys_unsafe,
                                                                 set(),
                                                                 cert_options)
t1 = time.time()
print(f"Time to certify path {t1-t0}")

In [ ]:
for s in stats:
    print(s.certified_safe())
    

In [ ]:
print(f"Total unsafe solver time = {sum([s.total_time_to_solve_progs()/20 for s in stats])}")

In [ ]:
print(sum([s.total_time_to_solve_progs()/20 for s in stats]))

In [ ]:
for s in stats:
    print(s.total_time_to_certify())
    print(s.total_time_to_solve_progs())
    print(s.total_time_building_progs())
    print()

In [ ]:
print(stats[0].total_time_to_certify_pair[0])
print(stats[0].time_to_build_prog)
print(stats[0].time_to_solve_prog[0])